In [1]:
from google.cloud import vision
import pyzbar.pyzbar as pyzbar
import io
import os
import numpy as np
import cv2
from PIL import ImageFont, ImageDraw, Image
import matplotlib.pyplot as plt
from flask import Flask, request, jsonify
from waitress import serve
import pymysql, logging
import re
import difflib
import random

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key/service-account-file.json'

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

dbinfo = {
    'host': '127.0.0.1',
    'username': 'root',
    'password': 'rlxo63',
    'database': 'capstone',
    'port': 3306,
}

# connect MySQL
try:
    conn = pymysql.connect(
        host=dbinfo['host'],
        user=dbinfo['username'],
        passwd=dbinfo['password'],
        db=dbinfo['database'],
        port=dbinfo['port'],
        use_unicode=True, charset='utf8'
    )
    cursor = conn.cursor()
except:
    logging.error("could not connect to rds")

In [ ]:
app = Flask(__name__)

def txt_barcode_extract(image_file):
    # barcode extract
    img_file = image_file.read()
    img_ary = np.asarray(bytearray(img_file), dtype = np.uint8)
    img = cv2.imdecode(img_ary, cv2.IMREAD_COLOR)
    code = pyzbar.decode(img)
    if code:
        code = code[0].data.decode("utf-8")
    # text extract
    image = vision.Image(content=img_file)
    response = client.text_detection(image=image)
    texts = response.text_annotations[0].description.replace('\n','')
    return code, texts
    
def find_ingredient(response):
    sql = "SELECT food_ingre from food_ingredients"
    cursor.execute(sql)
    result = cursor.fetchall()
    # 각 튜플을 문자열로 변환하고 이를 연결
    result_str = ' '.join(map(str, result))
    ingre = result_str.replace("(","").replace(")","").replace("\\","").replace(",","").replace("'","").replace("\"","")
    words = ingre.split('. ')
    words = list(set(words))
    matches = [ingredient for ingredient in words if re.search(ingredient, response)]
    return matches

def find_drug_text(response):
    min_len = 0
    sql = "SELECT DrugName from druginfo"
    cursor.execute(sql)
    result = cursor.fetchall()
    
    # 각 튜플을 문자열로 변환하고 이를 연결
    words = []
    for word in result:
        clean_word = re.sub(r'\([^)]*\)', '', word[0])  # 소괄호와 그 안의 내용 제거
        clean_word = re.sub(r'\d+밀리그램|\d+mg|\d+/\d+\.?\d*밀리그램|\d+\.?\d*밀리그램', '', clean_word)  # 용량 제거
        clean_word = re.sub(r'\d+\.?\d*\s*밀리그람|\d+mg|\d+/\d+\.?\d*\s*밀리그람|\d+\.?\d*\s*밀리그람', '', clean_word)
        words.append(clean_word.strip())  # 앞뒤 공백 제거
    
    words = list(set(words))
    matches = [ingredient for ingredient in words if re.search(re.escape(ingredient), response)]
    return matches

def find_caution(drug):
    sql = "SELECT Drug_caution_food FROM capstone.drug WHERE DrugName LIKE %s;"
    param = ("%" + drug + "%",)
    
    try:
        cursor.execute(sql, param)
        results = cursor.fetchall()
        
        # 결과 처리
        cautions = [row[0] for row in results]
        return cautions
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def extract_nutrients(response):
    # 띄어쓰기 제거
    text = response.replace(' ', '')
    # 정규 표현식을 사용하여 필요한 정보 추출
    patterns = {
        '칼로리': r'(\d+)kcal',
        '나트륨': r'나트륨(\d+)mg',
        '탄수화물': r'탄수화물(\d+)g',
        '당류': r'당류(\d+)g',
        '지방': r'지방(\d+\.\d+|\d+)g',
        '트랜스지방': r'트랜스지방(\d+)g',
        '포화지방': r'포화지방(\d+\.\d+|\d+)g',
        '콜레스테롤': r'콜레스테롤(-|\d+)mg',
        '단백질': r'단백질(\d+)g',
        '칼슘': r'칼슘(\d+)mg'
    }
    nutrients = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrients[key] = match.group(1)
    return nutrients

def find_name(name):
    sql = 'SELECT `item name` FROM capstone.food_ingredients WHERE `item name` LIKE %s;'
    param = ("%" + name + "%",)
    try:
        cursor.execute(sql, param)
        results = cursor.fetchall()
        
        # 결과 처리
        results = [row[0] for row in results]
        return results
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    

def recommend(aa, bb, cc): #할당량 채우면 1 아니면 0
#탄수화물 1
#당류 2
#단백질 3
#지방 4
#포화지방 5
#나트륨 6
#콜레스테롤 7
    start= "이(가) 풍부한 "
    end = "을(를) 드시면 좋을 것 같아요!"

    aaa="탄수화물"
    bbb="당류"
    ccc="단백질"
    ddd="지방"
    eee="포화지방"
    fff="나트륨"
    ggg="콜레스테롤"

    inp=sorted([aa, bb, cc])

    if(aa==0 and bb==0 and cc==0):
      return "그만 드시는걸 추천합니다."
    else:
      #하나만 부족
      if(inp[0]==0 and inp[1]==0):
        if(inp[2]==1):
          nutri=aaa
          food=random.choice(["쌀밥", "고구마", "오트밀", "파스타", "빵"])
        elif(inp[2]==2):
          nutri=bbb
          food=random.choice(["과일(사과, 바나나)", "꿀", "초콜릿", "음료수", "건포도"])
        elif(inp[2]==3):
          nutri=ccc
          food=random.choice( ["닭가슴살", "두부", "달걀", "콩", "그릭 요거트"])
        elif(inp[2]==4):
          nutri=ddd
          food=random.choice(["아보카도", "견과류", "올리브오일", "치즈", "초콜릿"])
        elif(inp[2]==5):
          nutri=eee
          food=random.choice(["저염 버터", "베이컨", "코코넛오일", "크림", "치즈케이크", "팜"])
        elif(inp[2]==6):
          nutri=fff
          food=random.choice(["소금", "간장", "피클", "가공된 육류(베이컨, 소시지)", "치즈"])
        elif(inp[2]==7):
          nutri=ggg
          food=random.choice(["새우", "달걀노른자", "게", "오징어", "치즈"])

      #둘 부족
      elif(inp[0]==0):
        if(inp[1]==1):
          if(inp[2]==2):
            nutri=aaa+", "+bbb
            food=random.choice( ["과일(바나나, 사과)", "건포도", "꿀"])
          elif(inp[2]==3):
            nutri=aaa+", "+ccc
            food=random.choice(["퀴노아", "병아리콩(후무스)", "렌틸콩", "그릭 요거트와 과일", "귀리와 우유"])
          elif(inp[2]==4):
            nutri=aaa+", "+ddd
            food=random.choice(["아보카도 토스트", "견과류와 말린 과일 믹스", "땅콩버터 샌드위치"])
          elif(inp[2]==5):
            nutri=aaa+", "+eee
            food=random.choice( ["버터를 바른 빵", "로제파스타", "코코넛 밀크와 밥"])
          elif(inp[2]==6):
            nutri=aaa+", "+fff
            food=random.choice(["소금빵", "볶음밥", "나초와 소금"])
          elif(inp[2]==7):
            nutri=aaa+", "+ggg
            food=random.choice(["달걀 프라이를 얹은 밥", "버터를 바른 토스트", "크림 파스타"])
        elif(inp[1]==2):
          if(inp[2]==3):
            nutri=bbb+", "+ccc
            food=random.choice( ["요거트와 꿀", "초콜릿 우유", "과일과 그릭 요거트"])
          elif(inp[2]==4):
            nutri=bbb+", "+ddd
            food=random.choice( ["초콜릿", "건포도와 견과류 믹스", "크림이 들어간 과일 샐러드"])
          elif(inp[2]==5):
            nutri=bbb+", "+eee
            food=random.choice( ["초콜릿 케이크", "크림 브륄레", "크림 커피"])
          elif(inp[2]==6):
            nutri=bbb+", "+fff
            food=random.choice(["꼬북칩", "팝콘", "카라멜 프레첼"],)
          elif(inp[2]==7):
            nutri=bbb+", "+ggg
            food=random.choice(["크림이 들어간 과일 샐러드", "초콜릿 케이크", "크림 파이"])
        elif(inp[1]==3):
          if(inp[2]==4):
            nutri=ccc+", "+ddd
            food=random.choice(["연어", "치즈와 견과류", "닭가슴살과 아보카도"])
          elif(inp[2]==5):
            nutri=ccc+", "+eee
            food=random.choice( ["베이컨과 달걀", "치즈 오믈렛", "스테이크"])
          elif(inp[2]==6):
            nutri=ccc+", "+fff
            food=random.choice(["베이컨과 달걀", "가공된 육류(햄, 소시지)", "간장 치킨"])
          elif(inp[2]==7):
            nutri=ccc+", "+ggg
            food=random.choice(["달걀", "간 요리", "새우"])
        elif(inp[1]==4):
          if(inp[2]==5):
            nutri=ddd+", "+eee
            food=random.choice( ["코코넛 오일로 조리한 음식", "치즈", "저염 버터"])
          elif(inp[2]==6):
            nutri=ddd+", "+fff
            food=random.choice(["치즈", "견과류에 소금을 첨가한 것", "버터"])
          elif(inp[2]==7):
            nutri=ddd+", "+ggg
            food=random.choice(["저염 버터", "크림", "지방이 많은 고기"])
        elif(inp[1]==5):
          if(inp[2]==6):
            nutri=eee+", "+fff
            food=random.choice( ["버터", "치즈", "가공된 육류(베이컨, 소시지)"],)
          elif(inp[2]==7):
            nutri=eee+", "+ggg
            food=random.choice(["저염 버터", "삼겹", "크림"])
        else:
          nutri=fff+", "+ggg
          food=random.choice(["간장 치킨", "가공된 육류(햄, 소시지)", "치즈"])

      #3가지 부족
      else:
        if(inp[0]==1):
          if(inp[1]==2):
            if(inp[2]==3):
              nutri=aaa+", "+bbb+", "+ccc
              food=random.choice( ["그릭 요거트와 꿀", "바나나와 땅콩버터", "오트밀과 베리, 견과류"])
            elif(inp[2]==4):
              nutri=aaa+", "+bbb+", "+ddd
              food=random.choice( ["초콜릿과 견과류", "땅콩버터와 바나나", "과일과 아보카도 샐러드"])
            elif(inp[2]==5):
              nutri=aaa+", "+bbb+", "+eee
              food=random.choice(["크림 브륄레", "초콜릿 케이크", "코코넛 밀크와 밥"])
            elif(inp[2]==6):
              nutri=aaa+", "+bbb+", "+fff
              food=random.choice(["카라멜 팝콘", "단짠 팝콘", "소금이 들어간 건포도 스낵"])
            elif(inp[2]==7):
              nutri=aaa+", "+bbb+", "+ggg
              food=random.choice( ["크림 파이", "초콜릿 케이크", "크림이 들어간 과일 샐러드"])
          elif(inp[1]==3):
            if(inp[2]==4):
              nutri=aaa+", "+ccc+", "+ddd
              food=random.choice(["퀴노아 샐러드", "연어와 퀴노아", "닭가슴살과 고구마"])
            elif(inp[2]==5):
              nutri=aaa+", "+ccc+", "+eee
              food=random.choice(["치킨 크림 파스타", "베이컨과 계란을 넣은 볶음밥", "치즈 오믈렛과 토스트"])
            elif(inp[2]==6):
              nutri=aaa+", "+ccc+", "+fff
              food=random.choice(["햄과 치즈 샌드위치", "소금이 들어간 병아리콩 샐러드", "간장 치킨과 밥"])
            elif(inp[2]==7):
              nutri=aaa+", "+ccc+", "+ggg
              food=random.choice(["새우 볶음밥", "달걀 프라이와 밥", "크림 치킨 파스타"])
          elif(inp[1]==4):
            if(inp[2]==5):
              nutri=aaa+", "+ddd+", "+eee
              food=random.choice( ["크림 파스타", "버터를 바른 토스트", "코코넛 밀크와 밥"])
            elif(inp[2]==6):
              nutri=aaa+", "+ddd+", "+fff
              food=random.choice(["치즈 피자", "나초와 치즈", "소금이 들어간 아보카도 토스트"])
            elif(inp[2]==7):
              nutri=aaa+", "+ddd+", "+ggg
              food=random.choice( ["크림 치즈 베이글", "새우 크림 파스타", "버터를 바른 토스트"])
          elif(inp[1]==5):
            if(inp[2]==6):
              nutri=aaa+", "+eee+", "+fff
              food=random.choice(["치즈 피자", "버터를 바른 빵", "가공된 육류와 빵"])
            elif(inp[2]==7):
              nutri=aaa+", "+eee+", "+ggg
              food=random.choice( ["치즈 오믈렛", "크림 치킨 파스타", "버터를 바른 빵"])
          else:
            nutri=aaa+", "+fff+", "+ggg
            food=random.choice(["치즈 오믈렛", "햄 샌드위치", "버터를 바른 빵"])
        elif(inp[0]==2):
          if(inp[1]==3):
            if(inp[2]==4):
              nutri=bbb+", "+ccc+", "+ddd
              food=random.choice(["요거트와 견과류", "초콜릿 프로틴 쉐이크", "땅콩버터와 과일"])
            elif(inp[2]==5):
              nutri=bbb+", "+ccc+", "+eee
              food=random.choice(["초콜릿 프로틴 쉐이크", "그릭 요거트와 꿀", "과일과 크림 치즈"])
            elif(inp[2]==6):
              nutri=bbb+", "+ccc+", "+fff
              food=random.choice( ["초콜릿 우유", "카라멜 팝콘", "가당된 치즈 스낵"])
            elif(inp[2]==7):
              nutri=bbb+", "+ccc+", "+ggg
              food=random.choice( ["크림 프로틴 쉐이크", "초콜릿 케이크", "크림이 들어간 과일 샐러드"])
          elif(inp[1]==4):
            if(inp[2]==5):
              nutri=bbb+", "+ddd+", "+eee
              food=random.choice(["초콜릿", "크림 브륄레", "코코넛 밀크 디저트"])
            elif(inp[2]==6):
              nutri=bbb+", "+ddd+", "+fff
              food=random.choice( ["카라멜 팝콘", "소금이 들어간 초콜릿", "단짠 스낵"])
            elif(inp[2]==7):
              nutri=bbb+", "+ddd+", "+ggg
              food=random.choice(["크림 파이", "초콜릿 케이크", "크림이 들어간 과일 샐러드"])
          elif(inp[1]==5):
            if(inp[2]==6):
              nutri=bbb+", "+eee+", "+fff
              food=random.choice( ["카라멜 팝콘", "소금이 들어간 초콜릿", "단짠 스낵"])
            elif(inp[2]==7):
              nutri=bbb+", "+eee+", "+ggg
              food=random.choice( ["초콜릿 케이크", "크림 브륄레", "크림 파이"])
          else:
            nutri=bbb+", "+fff+", "+ggg
            food=random.choice( ["카라멜 팝콘", "소금이 들어간 초콜릿", "단짠 스낵"])
        elif(inp[0]==3):
          if(inp[1]==4):
            if(inp[2]==5):
              nutri=ccc+", "+ddd+", "+eee
              food=random.choice(["베이컨", "치즈", "코코넛 오일로 조리한 음식"])
            elif(inp[2]==6):
              nutri=ccc+", "+ddd+", "+fff
              food=random.choice(["베이컨", "가공된 육류(햄, 소시지)", "치즈"])
            elif(inp[2]==7):
              nutri=ccc+", "+ddd+", "+ggg
              food=random.choice( ["달걀", "버터", "스테이크"])
          elif(inp[1]==5):
            if(inp[2]==6):
              nutri=ccc+", "+eee+", "+fff
              food=random.choice(["베이컨", "치즈", "햄", "소시지"])
            elif(inp[2]==7):
              nutri=ccc+", "+eee+", "+ggg
              food=random.choice(["스테이크", "베이컨", "달걀"])
          else:
            nutri=ccc+", "+fff+", "+ggg
            food=random.choice( ["햄", "소시지", "치즈"])
        elif(inp[0]==4):
          if(inp[1]==5):
            if(inp[2]==6):
              nutri=ddd+", "+eee+", "+fff
              food=random.choice( ["베이컨", "치즈", "가공된 육류(햄, 소시지)"])
            elif(inp[2]==7):
              nutri=ddd+", "+eee+", "+ggg
              food=random.choice(["버터", "치즈", "지방이 많은 고기"])
          else:
            nutri=ddd+", "+fff+", "+ggg
            food=random.choice(["치즈", "가공된 육류(햄, 소시지)", "버터"])
        else:
          nutri=eee+", "+fff+", "+ggg
          food=random.choice(["소시지", "치즈", "베이컨"])



      return nutri+start+food+end

@app.route('/recommend', methods=['POST'])
def recommend_server():
    data = request.get_json()
    if 'a' not in data:
        return 'No a info', 400
    elif 'b' not in data:
        return 'No b info', 400
    elif 'c' not in data:
        return 'No c info', 400
    a = data['a']
    b = data['b']
    c = data['c']
    result = recommend(a,b,c)
    return {'result': result}

@app.route('/extract_food', methods=['POST'])
def extract_food():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        code, response = txt_barcode_extract(file)
        matches = find_ingredient(response)
        if code:
            print(code)
            sql = "SELECT ItemName FROM barcode WHERE `DistributionBarcode` = %s"
            cursor.execute(sql,(code,))
            result = cursor.fetchone()
            if result:
                item_name = result['ItemName']
                sql = "SELECT food_ingre FROM food_ingredients WHERE `item name` = %s"
                cursor.execute(sql,(item_name,))
                result = cursor.fetchone()
                if result:
                    food_ingre = result['food_ingre']
                    return {'food_ingre': str(food_ingre), 'item_name': str(item_name)}, 200
                else:
                    return {'matches': str(matches)}, 200
            else:
                return {'matches': str(matches)}, 200
        else:
            return {'matches': str(matches)}, 200

@app.route('/drug_text', methods=['POST'])
def extract_drug():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        code, response = txt_barcode_extract(file)
        matches = find_drug_text(response)
        return {'matches': str(matches)}, 200
    
@app.route('/drug_food', methods=['POST'])
def drug_food():
    data = request.get_json()
    if 'drug' not in data:
        return 'No drug info', 400
    if 'food' not in data:
        return 'No food info', 400
    drugs = data['drug']
    foods = data['food']
    print(drugs)
    print(foods)
    matches = []

    for drug in drugs:
        cautions = find_caution(drug)
        for food in foods:
            for caution in cautions:
                # 단어 경계를 사용하여 정확한 단어 검색
                if re.search(r'\b' + re.escape(food) + r'\b', caution):
                    matches.append(food)

    if len(matches) == 0:
        matches.append('충돌 성분 없음')

    matches = list(set(matches))
    return {'matches': matches}, 200


@app.route('/food_ingre_text', methods=['POST'])
def food_ingre_name():
    data = request.get_json()
    if 'name' not in data:
        return {'error': 'No food name'}, 400  # 반환 형식 수정
    name = data['name']
    try:
        sql = 'SELECT food_ingre FROM food_ingredients WHERE `item name` = %s'
        cursor.execute(sql, (name,))
        result = cursor.fetchone()
        result = result[0].split('. ')

        # 결과를 적절한 형식으로 변환
        if result:
            return {'result': str(result)}, 200
        else:
            return {'result': 'No ingredients found'}, 404  # 추가적인 오류 처리
    except Exception as e:
        print(f"An error occurred: {e}")
        return {'error': str(e)}, 500
    

@app.route('/food_nutrients', methods=['POST'])
def food_nutrients():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        try:
            _, response = txt_barcode_extract(file)
            nutrients = extract_nutrients(response)
            return {'nutrients': nutrients}, 200
        except Exception as e:
            return {'error': str(e)}, 500
        
@app.route('/food_name', methods=['POST'])
def query_name():
    data = request.get_json()
    if 'name' not in data:
        return 'No name info', 400
    name = data['name']
    result = find_name(name)
    response = jsonify(result)
    response.headers["Content-Length"] = str(len(response.get_data()))
    response.headers["Connection"] = "close"
    return response

# @app.after_request
# def after_request(response):
#     response.headers["Connection"] = "close"
#     return response
        
# @app.route('/drugtxt', methods=['POST'])
# def upload_file():
#     if 'file' not in request.files:
#         return 'No file part', 400
#     file = request.files['file']
#     if file.filename == '':
#         return 'No selected file', 400


#     if file:
#         code, response = txt_barcode_extract(file)
#         return {'barcode': str(code), 'text': str(response)}, 200

if __name__ == '__main__':
    app.run(host = '0.0.0.0', port = '5000', debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.13:5000
Press CTRL+C to quit
210.94.223.50 - - [13/Jun/2024 03:19:32] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:19:51] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:19:59] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:20:07] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:20:37] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:20:43] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:20:56] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:21:00] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:21:09] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:21:17] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:22:46] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/

['로펜정', '삼진디아제팜정', '프로틴정', '이알정']
[]


210.94.223.50 - - [13/Jun/2024 03:23:33] "POST /extract_food HTTP/1.1" 200 -


8801007859743


...

210.94.223.50 - - [13/Jun/2024 03:23:54] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:24:16] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:24:31] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:26:18] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:26:43] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:26:52] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:27:46] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:27:48] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:27:53] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:27:57] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:28:03] "POST /food_nutrients HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:31:45] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:31:52] "POST /food_nutrients HTTP/1.1"

['로펜정', '삼진디아제팜정', '프로틴정', '이알정']
[]
['로펜정', '삼진디아제팜정', '프로틴정', '이알정']
[]
['로펜정', '삼진디아제팜정', '프로틴정', '이알정']
[]


210.94.223.50 - - [13/Jun/2024 03:36:48] "POST /drug_food HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:05] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:10] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:13] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:16] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:18] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:20] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:40] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:42] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:46] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:49] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:52] "POST /food_name HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:37:54] "POST /food_ingre_text HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 0

['로펜정', '삼진디아제팜정', '프로틴정', '이알정']
['플라보노이드', '카나우바왁스', '아세설팜칼륨', 'D-소비톨', '아스파탐', '치자청색소', '식품첨가물혼합제제', '카페인', '식품첨가물혼합제제', '식품첨가물혼합제제', '식품첨가물혼합제제', 'l-멘톨', '아라비아검', '말티톨시럽', '이소말트', '검베이스', '자일리톨', 'D-말티톨']
['로펜정', '삼진디아제팜정', '프로틴정', '이알정']
['플라보노이드', '카나우바왁스', '아세설팜칼륨', 'D-소비톨', '아스파탐', '치자청색소', '식품첨가물혼합제제', '카페인', '식품첨가물혼합제제', '식품첨가물혼합제제', '식품첨가물혼합제제', 'l-멘톨', '아라비아검', '말티톨시럽', '이소말트', '검베이스', '자일리톨', 'D-말티톨']


210.94.223.50 - - [13/Jun/2024 03:38:16] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:38:37] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:38:51] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:39:00] "POST /recommend HTTP/1.1" 200 -
210.94.223.50 - - [13/Jun/2024 03:39:25] "POST /recommend HTTP/1.1" 200 -


In [ ]:
code, response = txt_barcode_extract(io.open('drug.jpg','rb'))

In [ ]:
def find_ingredient(response):
    sql = "SELECT food_ingre from food_ingredients"
    cursor.execute(sql)
    result = cursor.fetchall()
    # 각 튜플을 문자열로 변환하고 이를 연결
    result_str = ' '.join(map(str, result))
    ingre = result_str.replace("(","").replace(")","").replace("\\","").replace(",","").replace("'","").replace("\"","")
    words = ingre.split('. ')
    words = list(set(words))
    matches = [ingredient for ingredient in words if re.search(ingredient, response)]
    return matches

def find_drug_text(response):
    min_len = 0
    sql = "SELECT DrugName from druginfo"
    cursor.execute(sql)
    result = cursor.fetchall()
    
    # 각 튜플을 문자열로 변환하고 이를 연결
    words = []
    for word in result:
        clean_word = re.sub(r'\([^)]*\)', '', word[0])  # 소괄호와 그 안의 내용 제거
        clean_word = re.sub(r'\d+밀리그램|\d+mg|\d+/\d+\.?\d*밀리그램|\d+\.?\d*밀리그램', '', clean_word)  # 용량 제거
        clean_word = re.sub(r'\d+\.?\d*\s*밀리그람|\d+mg|\d+/\d+\.?\d*\s*밀리그람|\d+\.?\d*\s*밀리그람', '', clean_word)
        words.append(clean_word.strip())  # 앞뒤 공백 제거
    
    words = list(set(words))
    matches = [ingredient for ingredient in words if re.search(re.escape(ingredient), response)]
    return matches

matches, response, words = find_drug_text(response)

In [ ]:
response

In [ ]:
matches

In [ ]:
from flask import Flask, request, jsonify
from waitress import serve

app = Flask(__name__)

@app.route('/bmr', methods=['POST'])
def calculate_bmr():
    data = request.get_json()
    gender = data.get('gender')
    weight = data.get('weight')
    height = data.get('height')
    age = data.get('age')
    act = data.get('act')
    bmr = BMR(gender, weight, height, age, act)
    return jsonify({'bmr': bmr})

if __name__ == "__main__":
    # Generate a self-signed certificate and key and replace 'cert.pem' and 'key.pem' with their paths
    app.run(ssl_context=('cert.pem', 'key.pem'))